# < 위도, 경도 기준으로 인근 도시 매칭하기 >

* 작업 목적 : 당사가 조사한 세계의 몇몇 관광명소들이 당사의 DB에 없는 도시에 있어서 매칭이 안된 곳이다. 그래서 DB에 있는 가장 가까운 도시를 우선 연결짓기로 하였고 이 작업을 수행한 것이다


* 전달받은 데이터셋 : 당사 DB에 있는 도시 정보 , 도시 매칭이 되지 않은 관광명소의 좌표 정보

In [1]:
import pandas as pd
# 마이셀럽스 도시 데이터
city_data = pd.read_excel("/Users/myhellebs/Downloads/mbl_city_data_update.xlsx")
# 매칭이 필요한 관광명소 데이터
match_list = pd.read_excel("/Users/myhellebs/Downloads/도시 매칭 필요 리스트.xlsx")

In [2]:
# 데이터셋 확인
match_list.head(3)
city_data

,cd_idx,cd_city_id,cd_name,cd_name_ko,cd_country,cd_latitude,cd_longitude,cd_timezone
0,1206763,-784833,Manama,마나마,bh,26.22350,50.5822,Asia/Qatar
1,1206765,-1204657,Arutanga,Arutanga,ck,-18.86410,-159.7980,Pacific/Rarotonga
2,1206766,-1204656,Arorangi,아로랑기,ck,-21.21670,-159.8170,Pacific/Rarotonga
3,1206767,900049229,Rarotonga,라로통가,ck,-21.22890,-159.7750,Pacific/Rarotonga
4,1206769,-1141980,Santa Cruz La Laguna,산타 크루스 라 라구나,gt,14.74420,-91.2073,America/Guatemala
...,...,...,...,...,...,...,...,...
17220,3180738,92707,Pasuljište,Pasuljište,rs,45.24170,19.7786,Europe/Belgrade
17221,3180885,121058,Staryy Poselok,Staryy Poselok,ua,46.64780,36.7594,Europe/Zaporozhye
17222,3180923,158541,Kojan,Kojan,kr,37.38280,126.7500,Asia/Seoul
17223,3180981,6164849,Maturín,Maturín,ve,9.75000,-63.1767,America/Caracas


## * 당사의 기준에 따른 거리 공식을 차용하였음(피타고라스 정리 이용)

## (두 지점의 직선거리)^2 = (x축의 거리차)^2 + (y축의 거리차)^2 

이 공식을 이용해 관광명소와 당사 DB 속 도시와의 직선거리가 가장 가까운 곳을 매칭

In [3]:
import pandas as pd

def search_city(lat, lng, city_data):
    
    # 위도값이 +-5이내
    lat_list = city_data[(city_data["cd_latitude"] > lat - 5) & (city_data["cd_latitude"] < lat + 5)].copy()
    # 위도값이 +-5이내이면서 경도값이 +-5이내
    data = lat_list[(lat_list["cd_longitude"] > lng - 5) & (lat_list["cd_longitude"] < lng + 5)]
    # 두 지점의 직선거리를 계산한 칼럼
    data['cal'] = ((lat - data["cd_latitude"]) ** 2 + (lng - data["cd_longitude"]) ** 2) ** (1/2)
    data = data.sort_values(by = 'cal')
                    
    try :
        match_city = data.iloc[0]['cd_name']
    
    except :
        match_city = None
                    
    return match_city

In [4]:
pd.set_option('mode.chained_assignment',  None)
match_list["매칭 도시"] = match_list.apply(lambda x : search_city(x["명소 위도"], x["명소 경도"], city_data), axis = 1)


In [5]:
# 최종 데이터 셋
match_list

,명소명,명소 주소,명소 위도,명소 경도,명소 타입,매칭 도시
0,Isola La Malghera,"28838 Stresa, 벌바노 쿠지오 오솔라 , 이탈리아",45.8984,8.52166,섬,Baveno
1,이즐라 데 라스 알메자스,"Isla de Las Almejas , 온두라스",13.3028,-87.61670,섬,La Unión
2,센터 아일랜드 비치,"Centre Island Beach , 캐나다 , 온타리오",43.6155,-79.37680,해변,Toronto
3,풀턴 체인 레이크스,"Fulton Chain Lakes , 미국 , 뉴욕",43.7499,-74.81630,호수,Old Forge
4,Anklamer Stadtbruch,"Dorfstraße 35, 17398 Bugewitz, 독일",53.8118,13.86120,자연보호구역,Karnin
...,...,...,...,...,...,...
245,포인트 오 로체 주립공원,"19 Camp Red Cloud Rd, Plattsburgh, NY 12901 미국",44.7854,-73.38190,주립공원,Saint-Bernard-de-Lacolle
246,Nordhavn Bassin,NaN,55.7057,12.59540,호수,København
247,Borromean Islands,"Isole Borromee , 벌바노 쿠지오 오솔라 , 이탈리아",45.9066,8.53717,제도,Pallanza
248,Sjoerd Ameland,"Strandweg 70, 9163 GN Nes, 네덜란드",53.4602,5.77862,음식점,Nes


## - 이후에 해당 작업을 기반으로 여행 컨텐츠를 기획하는 마케팅팀 인턴들에게 인수인계 해주어 일의 효율을 높였음

In [1]:
import pandas as pd
city_data = pd.read_excel("/Users/myhellebs/Downloads/mbl_city_data_update.xlsx")
#city_data = city_data.drop('cd_idx',axis=1).drop('cd_city_id',axis=1)

lat,lng = map(float, input('위도와 경도를 \',\'표시로 구분지어 입력하세요 :').split(','))

data = city_data[(city_data["cd_latitude"] > lat-10) & (city_data["cd_latitude"] < lat+10)].copy().sort_values(by = "cd_latitude")
final = data[(data["cd_longitude"] > lng-10) & (data["cd_longitude"] < lng+10)].sort_values(by = 'cd_latitude')
final["cal"] = ((lat-final["cd_latitude"])**2 + (lng - final["cd_longitude"])**2)**(1/2)
final[["cd_name", "cd_name_ko", "cd_latitude", "cd_longitude", "cd_country","cal"]].sort_values(by = "cal").head(5)

위도와 경도를 ','표시로 구분지어 입력하세요 :48.8606111,2.337644


,cd_name,cd_name_ko,cd_latitude,cd_longitude,cd_country,cal
5498,Paris,파리,48.8567,2.35148,fr,0.014378
5474,Saint-Ouen,생투앵,48.9117,2.33427,fr,0.051200
5550,Levallois-Perret,르발루아페레,48.8951,2.28730,fr,0.061025
5392,Vanves,벙브,48.8218,2.28653,fr,0.064179
5580,Issy-les-Moulineaux,이시레몰리노,48.8251,2.27346,fr,0.073353
